# Bind Mounts

### Introduction

In the last lesson, we saw how containers are temporary.  They are containers for processes and when the processes complete, the container terminates as well.  We saw this in the context of a blogging platform, Ghost.  And this seems odd, because if we have a blog, and we write a nice article, we don't want our blog posts to simply be destroyed if we need to shut down the website.

Instead, to persist data with our application, we have to separate out the persistence layer into another component.  There are two mechanisms to do this: bind mounts and volumes.  Here we'll learn about bind mounts and will save volumes for another lesson.

### Persistence with Docker

To see persistence with Docker, let's boot up a jupyter environment with docker.  We can pull, run, and map ports to the `jupyter/scipy-notebook` with the following command:

`docker run -p 8899:8888 jupyter/minimal-notebook`

If we do not already have the jupyter/scipy-notebook on our host computer, jupyter will pull it from jupyterhub.  Then jupyter will run the notebook, and map the host port 8899 to the container port 8888.


> This Docker image requires a login, or a token, by default.  Looking at the [documentation of the image](https://github.com/Paperspace/jupyter-docker-stacks/tree/master/scipy-notebook), we can disable authentication with the following: 

> `docker run -p 8888:8888 jupyter/scipy-notebook start-notebook.sh --NotebookApp.token=''`

Because we have mapped the ports, let's take a look visit our Jupyter notebook and create a new file called `index.ipynb`.  Then let's type in `pwd` to find the present working directory where the `index` notebook is stored in the container.

<img src="./scipy-volumes.png" width="60%"/>

Ok, so we see that the notebook is stored in `home/jovyan/work`.  Ok, now let's press `ctl+c` to shutdown our container.  Of course, we know that if we reboot the container, our notebook will disappear.

### Using Bind Mounts

One way to fix this is to use `bind mounts` in Docker.  From the [Docker documentation](https://docs.docker.com/storage/bind-mounts/):

> When you use a bind mount, a file or directory on the host machine is mounted into a container.  The file or directory is referenced by its full or relative path on the host machine. 

So with our Jupyter container, we can tell Docker that for the contents of the `/home/jovyan/work` container, to reference the contents of a specified working directory on the host.  This way, the contents won't be read and written inside of the container, but they will be read and written on a folder in our host, which is not so temporary.  Ok, let's do it.

**Creating data on the host**

First, just to get on the same page, on our host machine, let's:

1. Create a new directory called `practice`, and then navigate to that directory.
```
mkdir practice
cd practice
```

2. Then, let's add a text file called `special.md`.  
```
touch special.md
```

3. We can place the text `hello world` in the file, if we prefer, with: 

`echo "hello world" > special.md`

**Mapping data to the container**
Ok, now it's time to use our container, and map our `home/jovyan/work` folder to the `practice` folder on the host.  

Make sure you are in the `practice` folder on your machine, then run the following:

`(base) [practice (master)*]$ docker run -p 8899:8888 -v "$PWD":/home/jovyan/work jupyter/scipy-notebook`

This means to map the current folder (PWD stands for present working directory) into the folder `home/jovyan/work` in the container.  If we go to the `work` folder, this time we will find our `special.md` file.  We can also add a new file, and when we shutdown the container, we will see a new file in our folder.

<img src="./scipy-rw-volumes.png" width="80%"/> 

> Notice that the new file Hello.ipynb file that we created in the container, is available in our host's environment.

`(base) [practice (master)*]$ ls`
```
.ipynb_checkpoints Hello.ipynb        special.md
```

Cool, so we have just seen that by using bind mounts, we can both read and write files to our local disk by using the `-v` option when we run our container. 

This is a powerful concept.  By separating our storage of our notebooks or data from our container, we can evaluate our data in separate environments.  So, for example, with Jupyter, we can simply use a different Jupyter container or image to begin using our data.  Perhaps we have csv data that we want to use in with a Jupyter image sometime, and a database, a la postgres, other times.  We could use bind mounting to connect data on our local machine to a postgres containers sometimes and a jupyter contain when we prefer to use Jupyter.

### Summary

In this lesson, we saw one mechanism for persisting data inside of a container: using bind mounts.  With bind mounts, we map data from our local machine to inside of our container.  As we saw, the connection works both ways.  That is, any mounted data we change from inside of our container will be changed on our local machine, and any changes we make inside of the mounted folder will be seen inside of the container.  We mount our data by running a container with the `-v host_folder:container_folder` as in:

`docker run -v "$PWD":/home/jovyan/work jupyter/scipy-notebook`

### Resources

* [bind mounts](https://docs.docker.com/storage/bind-mounts/)
* [persistence in docker](https://docs.docker.com/storage/)